In [1]:
%matplotlib inline

# 優化模型參數

現在我們有了模型和數據，是時候通過優化數據上的參數來訓練、驗證和測試我們的模型了。訓練模型是一個迭代過程； 在每次迭代中（稱為 *epoch*），模型對輸出進行猜測，計算其猜測中的誤差（*loss*），收集誤差相對於其參數的導數（如我們在模塊中看到的） ，並使用梯度下降**優化**這些參數。有關此過程的更詳細演練，請查看 [來自 3Blue1Brown 的反向傳播](https://www.youtube.com/watch?v=tIeHLnjs5U8) 上的視頻。

##先決條件代碼

我們在 **Datasets & DataLoaders** 和 **Build Model** 上加載前面模塊的代碼



In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## 設置超參數

超參數是可調節的參數，可讓您控制模型優化過程。
不同的超參數值會影響模型訓練和收斂速度
（[閱讀更多]（https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html）關於超參數調整）

我們為訓練定義了以下超參數：
  - **Number of Epochs** - 迭代數據集的次數
  - **Batch Size** - 模型在每個 epoch 中看到的數據樣本數量
  - **Learning Rate** - 在每個批次/時期更新模型參數的程度。較小的值會導致學習速度變慢，而較大的值可能會導致訓練過程中出現不可預測的行為。

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## 添加優化循環

一旦我們設置了我們的超參數，我們就可以使用優化循環來訓練和優化我們的模型。每個
優化循環的迭代稱為 **epoch**。

每個時代由兩個主要部分組成：
 - **The Train Loop** - 迭代訓練數據集並嘗試收斂到最佳參數。
 - **The Validation/Test Loop** - 迭代測試數據集以檢查模型性能是否正在提高。

讓我們簡要地熟悉一下訓練循環中使用的一些概念。跳到
請參閱優化循環的“full-impl-label”。

### 添加損失函數

當提供一些訓練數據時，我們未經訓練的網絡可能無法給出正確的
回答。 **損失函數**衡量得到的結果與目標值的不同程度，
它是我們在訓練過程中想要最小化的損失函數。為了計算損失，我們做了一個
使用我們給定數據樣本的輸入進行預測，並將其與真實數據標籤值進行比較。

常見的損失函數包括用於回歸任務的 [nn.MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss)（均方誤差），以及
[nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss)（負對數似然）進行分類。
[nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss) 結合了`nn.LogSoftmax` 和`nn.NLLLoss`。

我們將模型的輸出 logits 傳遞給 `nn.CrossEntropyLoss`，它將對 logits 進行歸一化併計算預測誤差。

In [4]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

### Optimization pass

優化是在每個訓練步驟中調整模型參數以減少模型誤差的過程。 **優化算法**定義了這個過程是如何執行的（在這個例子中我們使用隨機梯度下降）。
所有優化邏輯都封裝在 ``optimizer`` 對像中。在這裡，我們使用 SGD 優化器； 此外，還有許多[不同的優化器](https://pytorch.org/docs/stable/optim.html)
在 PyTorch 中可用，例如 ADAM 和 RMSProp，它們更適用於不同類型的模型和數據。

我們通過註冊模型需要訓練的參數並傳入學習率超參數來初始化優化器。



In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

在訓練循環中，優化分三步進行：
  * 調用`optimizer.zero_grad()` 來重置模型參數的梯度。默認情況下漸變相加； 為了防止重複計算，我們在每次迭代時明確地將它們歸零。
  * 通過調用`loss.backwards()` 來反向傳播預測損失。 PyTorch 將損失的梯度存入 w.r.t. 每個參數。
  * 一旦我們有了梯度，我們就調用 ``optimizer.step()`` 來通過向後傳遞中收集的梯度來調整參數。

## 完整實現

我們定義了循環優化代碼的 `train_loop`，以及循環優化代碼的 `test_loop`
根據我們的測試數據評估模型的性能。

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):        
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

我們初始化損失函數和優化器，並將其傳遞給 `train_loop` 和 `test_loop`。
隨意增加 epochs 的數量以跟踪模型的改進性能。

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302778  [    0/60000]
loss: 2.297335  [ 6400/60000]
loss: 2.283023  [12800/60000]
loss: 2.272815  [19200/60000]
loss: 2.286411  [25600/60000]
loss: 2.254095  [32000/60000]
loss: 2.261106  [38400/60000]
loss: 2.249511  [44800/60000]
loss: 2.233091  [51200/60000]
loss: 2.205864  [57600/60000]
Test Error: 
 Accuracy: 43.4%, Avg loss: 0.034827 

Epoch 2
-------------------------------
loss: 2.225592  [    0/60000]
loss: 2.239306  [ 6400/60000]
loss: 2.188773  [12800/60000]
loss: 2.178016  [19200/60000]
loss: 2.227735  [25600/60000]
loss: 2.153194  [32000/60000]
loss: 2.171523  [38400/60000]
loss: 2.147953  [44800/60000]
loss: 2.107409  [51200/60000]
loss: 2.067630  [57600/60000]
Test Error: 
 Accuracy: 48.3%, Avg loss: 0.032724 

Epoch 3
-------------------------------
loss: 2.098490  [    0/60000]
loss: 2.124602  [ 6400/60000]
loss: 2.026889  [12800/60000]
loss: 2.017247  [19200/60000]
loss: 2.079707  [25600/60000]
loss: 1.978768  [32000/600

您可能已經註意到模型最初並不是很好（沒關係！）。嘗試運行循環以獲得更多的“epochs”或將“learning_rate”調整為更大的數字。也可能是我們選擇的模型配置可能不是此類問題的最佳配置（事實並非如此）。後面的課程將深入研究適用於視力問題的模型形狀。